## Exhustive search

In [22]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import itertools

def findMatchings(X):
    A = [0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4]
    n = len(A)
    combinations = []
    for i in range(n - 3):
        for j in range(i + 1, n - 2):
            l = j + 1
            r = n - 1
            while l < r:
                if A[i] + A[j] + A[l] + A[r] == X:
                    if [A[i], A[j], A[l], A[r]] not in combinations:
                        combinations.append([A[i], A[j], A[l], A[r]])
                    l += 1
                    r -= 1
                elif A[i] + A[j] + A[l] + A[r] < X:
                    l += 1
                else:
                    r -= 1

  # calculate permutations of each found combinations
  # print(combinations)

    permutations = []
    for item in combinations:
        temp = list(itertools.permutations(item))
        t = [list(e) for e in temp]
        permutations.extend(t)

    return list(map(list, set(map(tuple, permutations))))


In [26]:
results = findMatchings(4)

[[0, 2, 2, 0], [0, 0, 4, 0], [1, 0, 2, 1], [2, 2, 0, 0], [2, 0, 1, 1], [2, 0, 2, 0], [0, 2, 1, 1], [0, 0, 3, 1], [0, 0, 2, 2], [1, 0, 0, 3], [0, 3, 1, 0], [1, 0, 1, 2], [1, 1, 2, 0], [0, 1, 2, 1], [0, 1, 3, 0], [3, 0, 0, 1], [0, 4, 0, 0], [3, 0, 1, 0], [1, 2, 1, 0], [0, 3, 0, 1], [3, 1, 0, 0], [2, 0, 0, 2], [1, 1, 1, 1], [0, 2, 0, 2], [0, 1, 1, 2], [0, 0, 0, 4], [2, 1, 1, 0], [0, 0, 1, 3], [4, 0, 0, 0], [1, 2, 0, 1], [0, 1, 0, 3], [1, 1, 0, 2], [1, 0, 3, 0], [2, 1, 0, 1], [1, 3, 0, 0]]
35


In [47]:
from math import trunc
import random
import pandas as pd
import pickle as pkl
from tensorflow.keras.models import load_model

def generate_new_input():
    
    '''
    This functions creates a bimetallic chalcogenides by randomly choosing a transition metal, its comcentration
    and chalcogen element.  
    '''
    trans_metals = ['Ni', 'Co', 'Cr', 'Mn']
    chalcogens = ['S', 'Se', 'Te']

    y_all = np.linspace(6, 62.5, 1131)
    
    for A in trans_metals:
        for B in chalcogens:
            for y in y_all:
                x= 100-y  # percentage of Fe
                n = round(16*y/100) # Total number of substituted transition metalfor given substitution
                # call function to find all possible permutation of four integers that sums to n
                S_all = findMatchings(n)

                for S_item in S_all:
                    S1,S2,S3,S4 = S_item
#                     return [A,B,x,y,S1,S2,S3,S4]
                    sample = [A,B,x/100,y/100,S1,S2,S3,S4]
                    
                    # create new dataframe
                    cols = ['Fe','S1','S2','S3','S4','Ni','Co','Cr','Mn','Se','S','Te'] # list for cols to scale
                    df_new = pd.DataFrame(columns=cols)
                    data = [0]*12
                    df_new.loc[len(df_new)] = data
                    
                    transition_metal = sample[0]
                    chalcogen = sample[1]

                    df_new['Fe'] = sample[2]
                    df_new[transition_metal] = sample[3]
                    df_new[chalcogen] = 1
                    df_new['S1'] = sample[4]
                    df_new['S2'] = sample[5]
                    df_new['S3'] = sample[6]
                    df_new['S4'] = sample[7]

                    X_test = df_new.copy()

                    # load standard scaler
                    scaler = pkl.load(open('../models/standard_scaler/scaler.pkl', 'rb'))

                    # transform input data using saved standard scaler
                    cols_to_scale = ['Fe','S1','S2','S3','S4','Ni','Co','Cr','Mn','Se','S','Te'] # list for cols to scale
                    X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale]) # scale test data


                    # load base models
                    ANN_model = load_model("../models/base_models/ann.h5")
                    svr = pkl.load(open('../models/base_models/svr.pkl', 'rb'))
                    rf_reg = pkl.load(open('../models/base_models/rf_reg.pkl', 'rb'))
                    knn_reg = pkl.load(open('../models/base_models/knn_reg.pkl', 'rb'))
                    xgb_reg = pkl.load(open('../models/base_models/xgb_reg.pkl', 'rb'))
                    dt_reg = pkl.load(open('../models/base_models/dt_reg.pkl', 'rb'))

                    # load meta model
                    rf_meta_final = pkl.load(open('../models/meta_model/rf_meta_final.pkl', 'rb'))

                    # prepare new features based on base classifiers for test set
                    y_pred_test_ann = ANN_model.predict(X_test, verbose=0)
                    y_pred_test_ann = y_pred_test_ann.reshape(y_pred_test_ann.shape[0],)
                    y_pred_test_svm = svr.predict(X_test)
                    y_pred_test_rf = rf_reg.predict(X_test)
                    y_pred_test_knn = knn_reg.predict(X_test)
                    y_pred_test_xgb = xgb_reg.predict(X_test)
                    y_pred_test_dt = dt_reg.predict(X_test)

                    # get new features from base models
                    X_test_stacking = pd.DataFrame()
                    X_test_stacking['ann'] = y_pred_test_ann
                    X_test_stacking['svm'] = y_pred_test_svm
                    X_test_stacking['rf'] = y_pred_test_rf
                    X_test_stacking['knn'] = y_pred_test_knn
                    X_test_stacking['xgb'] = y_pred_test_xgb
                    X_test_stacking['dt'] = y_pred_test_dt

                    # apply new features to meta classifier
                    final_prediction = rf_meta_final.predict(X_test_stacking)

                   # write results to file
                    df_new['M_predicted'] = final_prediction
                    df_new.to_csv("output/exhaustive_search_results.csv", mode='a+', index=False, header= False)


                    

In [48]:
generate_new_input()

KeyboardInterrupt: 

In [17]:
y_all.shape

(1131,)

## Following code generates input data randomly for possible configuration of bimetal chalcogenides and calculates magnetic moment.

### Suresh and Dharmendra

In [1]:
from math import trunc
import random

def generate_new_input():
    
    '''
    This functions creates a bimetallic chalcogenides by randomly choosing a transition metal, its comcentration
    and chalcogen element.  
    '''
    # Enter transition metal
    # A = input("Enter transition metal: Ni, Co, Cr or Mn \n")

    trans_metal_dic = {1:'Ni', 2:'Co', 3:'Cr', 4:'Mn'}
    chalcogens_dic = {5:'S', 6: 'Se', 7:'Te'}

    # generate random number (0,1,2,3) and assign accordingly (Ni, Co, Cr, Mn)
    temp_1 = random.sample(range(1, 5), 1)[0]
    A = trans_metal_dic[temp_1]

    temp_2 = random.sample(range(5, 8), 1)[0]
    B = chalcogens_dic[temp_2]


    y = random.uniform(10,62.5) #Concentration of Substituted transition metal
    x= 100-y  # percentage of Fe



    n = 16*y/100 # Total number of substituted transition metalfor given substitution

    max_limit = 100*4/n # Calculate maximum nuber of transition metal on sites

## randomly fill sites (s1, s2, s3, s4) with atoms based on the % of sunstituted metal, No sites contains more than
## 4 atoms as this generalization is based on the FeS unit cell containing 16 Fe atoms, 4 sites and a site can hold only 
#4 atoms
    
    S1 = random.uniform(0, min(max_limit,100))

    rem_A = 100-S1 # Remaining percentage of transition metal
    
    n1 = n * rem_A / 100 # Remaining number of transition metal

    S2 = random.uniform(0, min(max_limit, rem_A))

    rem_A = 100 - S1 - S2
    n2 = n * rem_A / 100

    S3 = random.uniform(0, min(rem_A, max_limit))

    rem_A = 100-S1-S2-S3

    S4 = random.uniform(0, min(rem_A, max_limit))
    S4 = rem_A


    rem_A = 100-S1-S2-S3-S4 # check total % of Tran metal


    # Convert Percentage into numbers
    y_transformed = (y*16/100)/16
    x_transformed = 1-y_transformed
    S1_tranformed = S1*n/100
    S2_tranformed = S2*n/100
    S3_tranformed = S3*n/100
    S4_tranformed = S4*n/100

    if S4_tranformed > 4:      # As our model based on the unit cell FeS used in calculation, its a constraints
        # call model
#         print("Invalid input")
        return 0
    else:
        return [A,B,x_transformed,y_transformed,S1_tranformed,S2_tranformed,S3_tranformed,S4_tranformed]

    

In [2]:
def find_magnetic_moment(N_samples, output_file):
    
    """
    This funtion takes numbers of bimetallic chalcogeneides to investigate, and then caclulates magnetic moment
    of chalcogenides generated by 'generate_new_input()' using our final stacked machine learning model."""
    import pandas as pd
    import pickle as pkl
    from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
    from tensorflow.keras.models import load_model
    
    for iteration in range(N_samples):

        # create random dataset here
        cols = ['Fe','S1','S2','S3','S4','Ni','Co','Cr','Mn','Se','S','Te'] # list for cols to scale
        df_new = pd.DataFrame(columns=cols)
        data = [0]*12
        df_new.loc[len(df_new)] = data
        sample = generate_new_input()
        
        if sample == 0:
            continue
        
        transition_metal = sample[0]
        chalcogen = sample[1]

        df_new['Fe'] = sample[2]
        df_new[transition_metal] = sample[3]
        df_new[chalcogen] = 1
        df_new['S1'] = sample[4]
        df_new['S2'] = sample[5]
        df_new['S3'] = sample[6]
        df_new['S4'] = sample[7]

        X_test = df_new.copy()

        # load standard scaler
        scaler = pkl.load(open('../models/standard_scaler/scaler.pkl', 'rb'))

        # transform input data using saved standard scaler
        cols_to_scale = ['Fe','S1','S2','S3','S4','Ni','Co','Cr','Mn','Se','S','Te'] # list for cols to scale
        X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale]) # scale test data


        # load base models
        ANN_model = load_model("../models/base_models/ann.h5")
        svr = pkl.load(open('../models/base_models/svr.pkl', 'rb'))
        rf_reg = pkl.load(open('../models/base_models/rf_reg.pkl', 'rb'))
        knn_reg = pkl.load(open('../models/base_models/knn_reg.pkl', 'rb'))
        xgb_reg = pkl.load(open('../models/base_models/xgb_reg.pkl', 'rb'))
        dt_reg = pkl.load(open('../models/base_models/dt_reg.pkl', 'rb'))

        # load meta model
        rf_meta_final = pkl.load(open('../models/meta_model/rf_meta_final.pkl', 'rb'))

        # prepare new features based on base classifiers for test set
        y_pred_test_ann = ANN_model.predict(X_test, verbose=0)
        y_pred_test_ann = y_pred_test_ann.reshape(y_pred_test_ann.shape[0],)
        y_pred_test_svm = svr.predict(X_test)
        y_pred_test_rf = rf_reg.predict(X_test)
        y_pred_test_knn = knn_reg.predict(X_test)
        y_pred_test_xgb = xgb_reg.predict(X_test)
        y_pred_test_dt = dt_reg.predict(X_test)

        # get new features from base models
        X_test_stacking = pd.DataFrame()
        X_test_stacking['ann'] = y_pred_test_ann
        X_test_stacking['svm'] = y_pred_test_svm
        X_test_stacking['rf'] = y_pred_test_rf
        X_test_stacking['knn'] = y_pred_test_knn
        X_test_stacking['xgb'] = y_pred_test_xgb
        X_test_stacking['dt'] = y_pred_test_dt

        # apply new features to meta classifier
        final_prediction = rf_meta_final.predict(X_test_stacking)

       # write results to file
        df_new['M_predicted'] = final_prediction
        df_new.to_csv(output_file, mode='a+', index=False, header= False)
        

#### Finally, we can use the "find_magnetic_moment" to calculate magnetic moment as follows:

In [3]:
find_magnetic_moment(30000, 'output/test_random.csv')

### Following code is used to create Bimetallic Chalcogenides mannually by entering the values.

In [4]:
from math import trunc
# Enter transition metal
A = input("Enter transition metal: Ni, Co, Cr or Mn \n")

# Enter Chalcogen element
if A not in ['Ni', 'Co', 'Cr', 'Mn']:
    raise Exception("Invalid entry, choose one transition element")

B= input("Enter Chalcogen: S, Se or  Te \n")
if B not in ['S', 'Se', 'Te']:
    raise Exception("Invalid entry, choose one chalcogen element")

# Enter the percentage of substituted transition metal
y= int(input("Concentration of {} less than equals to 62.5% \n".format(A)))
x= 100-y  # percentage of Fe

if y> 62.5:
    raise Exception(" Enter y less than 62.5")

n = 16*y/100 # Total number of substituted transition metal
max_limit = 100*4/n # Calculate maximum nuber of transition metal on sites

# Assign the percentage of transition metal at the atomic site S1
S1= eval(input("Enter % of {} for atomic site S1: \n S1 should be <= {}% \n".format(A,trunc(max_limit))))

if S1> max_limit:
    raise Exception(" Value out of range: please check S1's maximum value")

rem_A = 100-S1 # Remaining percentage of transition metal
n1 = n*rem_A/100 # Remaining number of transition metal
print()
# Assign the percentage of transition metal at the atomic site S2
S2= eval(input("Enter % of {} for atomic site S2: \n S2 should be <= {}% \n"
               "Now you have only {}% of {} left \n".format(A,trunc(max_limit),rem_A, A)))

rem_A = 100-S1-S2
n2 = n*rem_A/100
if rem_A < 0 or S2 > trunc(max_limit):
    raise Exception(" Value out of range: please enter appropriate value")


# Assign the percentage of transition metal at the atomic site S3
S3 = eval(input("Enter % of {} for atomic site S3: \n S3 should be <= {}% \n"
                "Now you have only {}% of {} left \n".format(A, trunc(max_limit), rem_A, A)))

rem_A = 100-S1-S2-S3
if rem_A <0 or S2 > trunc(max_limit):
    raise Exception(" Value out of range: please enter appropriate value")


# Assign the percentage of transition metal at the atomic site S4
S4= eval(input("Enter % of {} for atomic site S4: \n S4 should be <= {}% \n"
               "Now you have only {}% of {} left".format(A,trunc(max_limit),rem_A, A)))

rem_A = 100-S1-S2-S3-S4
if rem_A <0 or S2 > trunc(max_limit):
    raise Exception(" Value out of range: please enter appropriate value")

# Convert Percentage into numbers
y_transformed = (y*16/100)/16
x_transformed = 1-y_transformed
S1_tranformed = S1*n/100
S2_tranformed = S2*n/100
S3_tranformed = S3*n/100
S4_tranformed = S4*n/100
print('=====================================')

print("Transition metal: ", A,"\nChalcogen: ", B, "\nx: ", x_transformed, "\ny: ", y_transformed,
      "\nS1: ", S1_tranformed, "\nS2: ", S2_tranformed, "\nS3: ", S3_tranformed, "\nS4:", S4_tranformed)


Enter transition metal: Ni, Co, Cr or Mn 
Ni
Enter Chalcogen: S, Se or  Te 
S
Concentration of Ni less than equals to 62.5% 
34
Enter % of Ni for atomic site S1: 
 S1 should be <= 73% 
65

Enter % of Ni for atomic site S2: 
 S2 should be <= 73% 
Now you have only 35% of Ni left 
23
Enter % of Ni for atomic site S3: 
 S3 should be <= 73% 
Now you have only 12% of Ni left 
7
Enter % of Ni for atomic site S4: 
 S4 should be <= 73% 
Now you have only 5% of Ni left5
Transition metal:  Ni 
Chalcogen:  S 
x:  0.6599999999999999 
y:  0.34 
S1:  3.536 
S2:  1.2512 
S3:  0.3808 
S4: 0.272
